In [2]:
# !pip install catboost

You should consider upgrading via the 'c:\users\yc yoon\desktop\githup\dacon\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.svm import LinearSVR, SVR
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
kf = KFold(n_splits = 15, shuffle = True, random_state = 42) # 보통 5, 10번 정도 한다.
# 여기선 15번, 셔플, 시드값42

#### LeaveOneOut을 사용했지만 KFold 보다 성능이 좋지 않아 15Fold를 사용했습니다.

In [5]:
train = pd.read_csv("../data/iris_train.csv")
test = pd.read_csv("../data/iris_test.csv")

In [6]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [7]:
test.head()

,id,species,sepal length (cm),petal length (cm)
0,0,setosa,5.4,1.7
1,1,setosa,5.7,1.5
2,2,setosa,5.3,1.5
3,3,setosa,5.1,1.9
4,4,virginica,6.0,4.8


In [8]:
scaler = MinMaxScaler() # 하나의 인자가 너무 큰영향을 주는것을 방지한다. 

#### Scaling 기법 중 MinMaxScaling이 성능이 좋아 사용했습니다.
- 학습 데이터의 분포를 학습한 scaler를 적용하기 위해 test 데이터에 대해서는 transform만 취해줍니다.

In [9]:
train.iloc[:, 2:4] = scaler.fit_transform(train.iloc[:, 2:4]) # 스케일 전환
test.iloc[:, 2:] = scaler.transform(test.iloc[:, 2:]) # 스케일 전환

In [10]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,0.029412,0.051724,2.9,0.2
1,1,versicolor,0.617647,0.586207,3.2,1.5
2,2,virginica,0.558824,0.637931,2.8,1.8
3,3,virginica,0.852941,0.862069,3.6,2.5
4,4,setosa,0.176471,0.051724,3.0,0.2


In [11]:
test.head()

,id,species,sepal length (cm),petal length (cm)
0,0,setosa,0.323529,0.103448
1,1,setosa,0.411765,0.068966
2,2,setosa,0.294118,0.068966
3,3,setosa,0.235294,0.137931
4,4,virginica,0.500000,0.637931


In [12]:
train['sw'] = train['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2})
test['sw'] = test['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2})
# 새로운 컬럼을 만들어 map을통해 값을 넣는다.

In [13]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),sw
0,0,setosa,0.029412,0.051724,2.9,0.2,3
1,1,versicolor,0.617647,0.586207,3.2,1.5,1
2,2,virginica,0.558824,0.637931,2.8,1.8,2
3,3,virginica,0.852941,0.862069,3.6,2.5,2
4,4,setosa,0.176471,0.051724,3.0,0.2,3


#### Sepal width (cm)을 예측할 때 target encoding을 위해 species별 sepal width (cm)의 평균을 기준으로 랭크를 생성했습니다.

In [14]:
train['pw'] = train['species'].map({'setosa' : 3, 'versicolor' : 2, 'virginica' : 1})
test['pw'] = test['species'].map({'setosa' : 3, 'versicolor' : 2, 'virginica' : 1})

In [15]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),sw,pw
0,0,setosa,0.029412,0.051724,2.9,0.2,3,3
1,1,versicolor,0.617647,0.586207,3.2,1.5,1,2
2,2,virginica,0.558824,0.637931,2.8,1.8,2,1
3,3,virginica,0.852941,0.862069,3.6,2.5,2,1
4,4,setosa,0.176471,0.051724,3.0,0.2,3,3


#### Petal width (cm)을 예측할 때 target encoding을 위해 species별 petal width (cm)의 평균을 기준으로 랭크를 생성했습니다.

In [16]:
X1 = train.drop(['id', 'species', 'sepal width (cm)', 'petal width (cm)', 'pw'], axis = 1)
X2 = train.drop(['id', 'species', 'sepal width (cm)', 'petal width (cm)', 'sw'], axis = 1)
y1 = train['sepal width (cm)']
y2 = train['petal width (cm)']

target1 = test[X1.columns]
target2 = test[X2.columns]

In [20]:
X1

,sepal length (cm),petal length (cm),sw
0,0.029412,0.051724,3
1,0.617647,0.586207,1
2,0.558824,0.637931,2
3,0.852941,0.862069,2
4,0.176471,0.051724,3
...,...,...,...
70,0.647059,0.603448,1
71,0.382353,0.431034,1
72,0.558824,0.586207,1
73,0.176471,0.379310,1


In [21]:
X2

,sepal length (cm),petal length (cm),pw
0,0.029412,0.051724,3
1,0.617647,0.586207,2
2,0.558824,0.637931,1
3,0.852941,0.862069,1
4,0.176471,0.051724,3
...,...,...,...
70,0.647059,0.603448,2
71,0.382353,0.431034,2
72,0.558824,0.586207,2
73,0.176471,0.379310,2


In [19]:
target1

,sepal length (cm),petal length (cm),sw
0,0.323529,0.103448,3
1,0.411765,0.068966,3
2,0.294118,0.068966,3
3,0.235294,0.137931,3
4,0.500000,0.637931,2
...,...,...,...
70,0.441176,0.500000,1
71,0.029412,0.034483,3
72,0.588235,0.775862,2
73,0.235294,0.327586,1


#### 최종적으로 2개의 데이터셋을 구성했습니다.
- sepal width (cm) 용
- petal width (cm) 용
***
### Linear Regression

In [23]:
print(kf)

KFold(n_splits=15, random_state=42, shuffle=True)


In [25]:
print(kf.split(X1, y1))

<generator object _BaseKFold.split at 0x0000019BB5C1C120>


In [27]:
print(zip(kf.split(X1, y1)))

In [32]:
print(list(zip(kf.split(X1, y1))))
print(list(zip(kf.split(X1, y1)))[0])


[((array([ 1,  2,  3,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74]), array([ 0,  4, 10, 35, 63])),), ((array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 65, 66, 67, 68, 70, 71, 72,
       73, 74]), array([12, 28, 61, 64, 69])),), ((array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36,
       37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,

In [33]:
lr_pred1 = np.zeros((target1.shape[0]))
print(lr_pred1)
vm_list = []
for i, idx in enumerate(zip(kf.split(X1, y1))) :
    tr_x, tr_y = X1.iloc[idx[0][0]], y1.iloc[idx[0][0]] # 훈련 셋
    val_x, val_y = X1.iloc[idx[0][1]], y1.iloc[idx[0][1]] # 평가 셋
    
    ### fitting
    lr = LinearRegression(normalize = True)
    lr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = lr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)
    
    ### prediction
    fold_pred = lr.predict(target1) / 15
    lr_pred1 += fold_pred
    
print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')
print(vm_list)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
15 FOLD 평균 MAE = 0.23283224802305147
[0.31796437811660383, 0.20538774536829446, 0.1041721452506394, 0.24485439182040247, 0.24257442243031804, 0.06521778854931322, 0.2665538090537861, 0.19095197909138167, 0.25496306296653426, 0.22507016941053318, 0.14674690595280282, 0.35554409167245415, 0.2830875101582926, 0.3782149460872882, 0.2111803744171273]


In [34]:
lr_pred2 = np.zeros((target2.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X2, y2))) :
    
    tr_x, tr_y = X2.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X2.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    ### fitting
    lr = LinearRegression()
    lr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = lr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)
    
    ### prediction
    fold_pred = lr.predict(target2) / 15
    lr_pred2 += fold_pred
    
print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

15 FOLD 평균 MAE = 0.14261988003428835


***
### SVR

In [35]:
svr_pred1 = np.zeros((target1.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X1, y1))) :

    tr_x, tr_y = X1.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X1.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    ### fitting
    svr = SVR(C = 5) # C는 Regularization 정도이다.
    svr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = svr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)
    
    ### prediction
    fold_pred = svr.predict(target1) / 15
    svr_pred1 += fold_pred

print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

15 FOLD 평균 MAE = 0.21420952021623502


In [36]:
svr_pred2 = np.zeros((target2.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X2, y2))) :
    tr_x, tr_y = X2.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X2.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    ### fitting
    svr = SVR(C = 5)
    svr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = svr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)
    
    ### prediction
    fold_pred = svr.predict(target2) / 15
    svr_pred2 += fold_pred

print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

15 FOLD 평균 MAE = 0.138913737472983


***
### ExtraTreeRegressor

In [37]:
etr_pred1 = np.zeros((target1.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X1, y1))) :

    tr_x, tr_y = X1.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X1.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    ### fitting
    etr = ExtraTreesRegressor(random_state = 42, n_estimators = 15, max_depth = 3)
    etr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = etr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)

    
    ### prediction
    fold_pred = etr.predict(target1) / 15
    etr_pred1 += fold_pred

print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

15 FOLD 평균 MAE = 0.23058993733356375


In [38]:
etr_pred2 = np.zeros((target2.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X2, y2))) :

    tr_x, tr_y = X2.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X2.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    ###fitting
    etr = ExtraTreesRegressor(random_state = 42, n_estimators = 15, max_depth = 3)
    etr.fit(tr_x, tr_y)
    
    ### validation
    val_pred = etr.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)

    
    ### prediction
    fold_pred = etr.predict(target2) / 15
    etr_pred2 += fold_pred

print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

15 FOLD 평균 MAE = 0.1462207498576399


***

### CatBoost

In [39]:
cb_pred1 = np.zeros((target1.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X1, y1))) :

    tr_x, tr_y = X1.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X1.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    cb = CatBoostRegressor(iterations = 5000, learning_rate = 0.1, max_depth = 3, loss_function = 'MAE')
    
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 1000)
    
    ### validation
    val_pred = cb.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)

    ### prediction
    fold_pred = cb.predict(target1) / 15
    cb_pred1 += fold_pred

print(f'15 FOLD 평균 MAE = {np.mean(vm_list)}')

0:	learn: 0.2748562	test: 0.2748562	test1: 0.3279981	best: 0.3279981 (0)	total: 130ms	remaining: 10m 48s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.2041686452
bestIteration = 87

Shrink model to first 88 iterations.
0:	learn: 0.2755704	test: 0.2755704	test1: 0.3139988	best: 0.3139988 (0)	total: 274us	remaining: 1.37s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.2333379438
bestIteration = 34

Shrink model to first 35 iterations.
0:	learn: 0.2877132	test: 0.2877132	test1: 0.1719992	best: 0.1719992 (0)	total: 336us	remaining: 1.68s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.0880096839
bestIteration = 91

Shrink model to first 92 iterations.
0:	learn: 0.2839989	test: 0.2839989	test1: 0.2619991	best: 0.2619991 (0)	total: 343us	remaining: 1.72s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.2144369068
bestIteration = 24

Shrink model to first 25 iterations.
0:	learn: 0.2822847	test: 0.2822847	te

In [40]:
cb_pred2 = np.zeros((target2.shape[0]))
vm_list = []
for i, idx in enumerate(zip(kf.split(X2, y2))) :

    tr_x, tr_y = X2.iloc[idx[0][0]], y2.iloc[idx[0][0]]
    val_x, val_y = X2.iloc[idx[0][1]], y2.iloc[idx[0][1]]
    
    ### fitting
    cb = CatBoostRegressor(iterations = 5000, learning_rate = 0.2, max_depth = 3, loss_function = 'MAE')
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 1000)
    
    ### validation
    val_pred = cb.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)

    ### prediction
    fold_pred = cb.predict(target2) / 15
    cb_pred2 += fold_pred

print(f'5 FOLD 평균 MAE = {np.mean(vm_list)}')

0:	learn: 0.4625704	test: 0.4625704	test1: 0.7239986	best: 0.7239986 (0)	total: 331us	remaining: 1.66s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.07832421311
bestIteration = 23

Shrink model to first 24 iterations.
0:	learn: 0.4634277	test: 0.4634277	test1: 0.6199996	best: 0.6199996 (0)	total: 313us	remaining: 1.57s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.1945763624
bestIteration = 28

Shrink model to first 29 iterations.
0:	learn: 0.4737134	test: 0.4737134	test1: 0.3599992	best: 0.3599992 (0)	total: 413us	remaining: 2.07s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.04103778319
bestIteration = 208

Shrink model to first 209 iterations.
0:	learn: 0.4808564	test: 0.4808564	test1: 0.2600000	best: 0.2600000 (0)	total: 307us	remaining: 1.54s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.1408439539
bestIteration = 60

Shrink model to first 61 iterations.
0:	learn: 0.4911419	test: 0.4911419	

***
### DNN

- KFold를 통한 앙상블
- 각 Fold별 best 모델 저장 및 test 예측 데이터에 load 후 사용
- callbacks 활용

In [41]:
train = pd.read_csv("../data/iris_train.csv")
test = pd.read_csv("../data/iris_test.csv")

In [42]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [43]:
print(train)

    id  sepal length (cm)  petal length (cm)  sepal width (cm)  \
0    0                4.4                1.4               2.9   
1    1                6.4                4.5               3.2   
2    2                6.2                4.8               2.8   
3    3                7.2                6.1               3.6   
4    4                4.9                1.4               3.0   
..  ..                ...                ...               ...   
70  70                6.5                4.6               2.8   
71  71                5.6                3.6               2.9   
72  72                6.2                4.5               2.2   
73  73                4.9                3.3               2.4   
74  74                6.9                4.9               3.1   

    petal width (cm)  species_setosa  species_versicolor  species_virginica  
0                0.2               1                   0                  0  
1                1.5               0               

In [44]:
X = train.drop(['id', 'sepal width (cm)', 'petal width (cm)'], axis = 1)
y1 = train['sepal width (cm)']
y2 = train['petal width (cm)']

target = test[X.columns]

In [45]:
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
target = pd.DataFrame(scaler.transform(target), columns = X.columns)

In [47]:
!pip install tensorflow

  Using cached tensorflow-2.6.0-cp39-cp39-win_amd64.whl (423.3 MB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached grpcio-1.41.1-cp39-cp39-win_amd64.whl (3.2 MB)
  Using cached clang-5.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached h5py-3.1.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached termcolor-1.1.0.tar.gz (3.

You should consider upgrading via the 'c:\users\yc yoon\desktop\githup\dacon\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [48]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [50]:
dnn_pred1 = np.zeros((target.shape[0]))
print(dnn_pred1)
print(len(dnn_pred1))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
75


In [51]:
for tr_idx, val_idx in kf.split(X, y1) :
    tr_x, tr_y = X.iloc[tr_idx], y1.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y1.iloc[val_idx]

    model = Sequential()
    model.add(Dense(512, activation = 'swish'))
    model.add(Dense(256, activation = 'swish'))
    model.add(Dropout(.2))
    model.add(Dense(128, activation = 'swish'))
    model.add(Dense(64, activation = 'swish'))
    model.add(Dropout(.2))
    model.add(Dense(8, activation = 'swish'))
    model.add(Dense(1, activation = 'linear'))

    model.compile(optimizer = Adam(learning_rate = .0003), loss = 'mse')

    es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 300, verbose = 100)
    mc = ModelCheckpoint('best_dnn.h5', mode = 'min', monitor = 'val_loss', save_best_only = True, verbose = 100)

    history = model.fit(tr_x, tr_y, epochs = 25000, verbose = 100, batch_size = 16, callbacks = [es, mc], validation_data = (val_x, val_y))
    best_model = load_model('best_dnn.h5')

    fold_pred = best_model.predict(target) / 15

    dnn_pred1 += [p[0] for p in fold_pred]

Epoch 1/25000

Epoch 00001: val_loss improved from inf to 7.02835, saving model to best_dnn.h5
Epoch 2/25000

Epoch 00002: val_loss improved from 7.02835 to 6.70286, saving model to best_dnn.h5
Epoch 3/25000

Epoch 00003: val_loss improved from 6.70286 to 6.28372, saving model to best_dnn.h5
Epoch 4/25000

Epoch 00004: val_loss improved from 6.28372 to 5.73978, saving model to best_dnn.h5
Epoch 5/25000

Epoch 00005: val_loss improved from 5.73978 to 5.07688, saving model to best_dnn.h5
Epoch 6/25000

Epoch 00006: val_loss improved from 5.07688 to 4.29232, saving model to best_dnn.h5
Epoch 7/25000

Epoch 00007: val_loss improved from 4.29232 to 3.31420, saving model to best_dnn.h5
Epoch 8/25000

Epoch 00008: val_loss improved from 3.31420 to 2.13752, saving model to best_dnn.h5
Epoch 9/25000

Epoch 00009: val_loss improved from 2.13752 to 1.11685, saving model to best_dnn.h5
Epoch 10/25000

Epoch 00010: val_loss improved from 1.11685 to 0.99739, saving model to best_dnn.h5
Epoch 11/2500

Epoch 118/25000

Epoch 00118: val_loss did not improve from 0.08582
Epoch 119/25000

Epoch 00119: val_loss did not improve from 0.08582
Epoch 120/25000

Epoch 00120: val_loss did not improve from 0.08582
Epoch 121/25000

Epoch 00121: val_loss did not improve from 0.08582
Epoch 122/25000

Epoch 00122: val_loss did not improve from 0.08582
Epoch 123/25000

Epoch 00123: val_loss did not improve from 0.08582
Epoch 124/25000

Epoch 00124: val_loss did not improve from 0.08582
Epoch 125/25000

Epoch 00125: val_loss did not improve from 0.08582
Epoch 126/25000

Epoch 00126: val_loss did not improve from 0.08582
Epoch 127/25000

Epoch 00127: val_loss did not improve from 0.08582
Epoch 128/25000

Epoch 00128: val_loss did not improve from 0.08582
Epoch 129/25000

Epoch 00129: val_loss did not improve from 0.08582
Epoch 130/25000

Epoch 00130: val_loss did not improve from 0.08582
Epoch 131/25000

Epoch 00131: val_loss did not improve from 0.08582
Epoch 132/25000

Epoch 00132: val_loss did not i

Epoch 242/25000

Epoch 00242: val_loss did not improve from 0.08068
Epoch 243/25000

Epoch 00243: val_loss did not improve from 0.08068
Epoch 244/25000

Epoch 00244: val_loss did not improve from 0.08068
Epoch 245/25000

Epoch 00245: val_loss did not improve from 0.08068
Epoch 246/25000

Epoch 00246: val_loss did not improve from 0.08068
Epoch 247/25000

Epoch 00247: val_loss did not improve from 0.08068
Epoch 248/25000

Epoch 00248: val_loss did not improve from 0.08068
Epoch 249/25000

Epoch 00249: val_loss did not improve from 0.08068
Epoch 250/25000

Epoch 00250: val_loss did not improve from 0.08068
Epoch 251/25000

Epoch 00251: val_loss did not improve from 0.08068
Epoch 252/25000

Epoch 00252: val_loss did not improve from 0.08068
Epoch 253/25000

Epoch 00253: val_loss did not improve from 0.08068
Epoch 254/25000

Epoch 00254: val_loss did not improve from 0.08068
Epoch 255/25000

Epoch 00255: val_loss did not improve from 0.08068
Epoch 256/25000

Epoch 00256: val_loss did not i

Epoch 367/25000

Epoch 00367: val_loss did not improve from 0.05873
Epoch 368/25000

Epoch 00368: val_loss did not improve from 0.05873
Epoch 369/25000

Epoch 00369: val_loss did not improve from 0.05873
Epoch 370/25000

Epoch 00370: val_loss did not improve from 0.05873
Epoch 371/25000

Epoch 00371: val_loss did not improve from 0.05873
Epoch 372/25000

Epoch 00372: val_loss did not improve from 0.05873
Epoch 373/25000

Epoch 00373: val_loss did not improve from 0.05873
Epoch 374/25000

Epoch 00374: val_loss did not improve from 0.05873
Epoch 375/25000

Epoch 00375: val_loss did not improve from 0.05873
Epoch 376/25000

Epoch 00376: val_loss did not improve from 0.05873
Epoch 377/25000

Epoch 00377: val_loss did not improve from 0.05873
Epoch 378/25000

Epoch 00378: val_loss did not improve from 0.05873
Epoch 379/25000

Epoch 00379: val_loss did not improve from 0.05873
Epoch 380/25000

Epoch 00380: val_loss did not improve from 0.05873
Epoch 381/25000

Epoch 00381: val_loss did not i

Epoch 497/25000

Epoch 00497: val_loss did not improve from 0.05873
Epoch 498/25000

Epoch 00498: val_loss did not improve from 0.05873
Epoch 499/25000

Epoch 00499: val_loss did not improve from 0.05873
Epoch 500/25000

Epoch 00500: val_loss did not improve from 0.05873
Epoch 501/25000

Epoch 00501: val_loss did not improve from 0.05873
Epoch 502/25000

Epoch 00502: val_loss did not improve from 0.05873
Epoch 503/25000

Epoch 00503: val_loss did not improve from 0.05873
Epoch 504/25000

Epoch 00504: val_loss did not improve from 0.05873
Epoch 505/25000

Epoch 00505: val_loss did not improve from 0.05873
Epoch 506/25000

Epoch 00506: val_loss did not improve from 0.05873
Epoch 507/25000

Epoch 00507: val_loss did not improve from 0.05873
Epoch 508/25000

Epoch 00508: val_loss did not improve from 0.05873
Epoch 509/25000

Epoch 00509: val_loss did not improve from 0.05873
Epoch 510/25000

Epoch 00510: val_loss did not improve from 0.05873
Epoch 511/25000

Epoch 00511: val_loss did not i

...

Epoch 126/25000

Epoch 00126: val_loss did not improve from 0.09723
Epoch 127/25000

Epoch 00127: val_loss did not improve from 0.09723
Epoch 128/25000

Epoch 00128: val_loss did not improve from 0.09723
Epoch 129/25000

Epoch 00129: val_loss did not improve from 0.09723
Epoch 130/25000

Epoch 00130: val_loss did not improve from 0.09723
Epoch 131/25000

Epoch 00131: val_loss did not improve from 0.09723
Epoch 132/25000

Epoch 00132: val_loss did not improve from 0.09723
Epoch 133/25000

Epoch 00133: val_loss did not improve from 0.09723
Epoch 134/25000

Epoch 00134: val_loss did not improve from 0.09723
Epoch 135/25000

Epoch 00135: val_loss did not improve from 0.09723
Epoch 136/25000

Epoch 00136: val_loss did not improve from 0.09723
Epoch 137/25000

Epoch 00137: val_loss did not improve from 0.09723
Epoch 138/25000

Epoch 00138: val_loss did not improve from 0.09723
Epoch 139/25000

Epoch 00139: val_loss did not improve from 0.09723
Epoch 140/25000

Epoch 00140: val_loss did not i

Epoch 253/25000

Epoch 00253: val_loss did not improve from 0.09723
Epoch 254/25000

Epoch 00254: val_loss did not improve from 0.09723
Epoch 255/25000

Epoch 00255: val_loss did not improve from 0.09723
Epoch 256/25000

Epoch 00256: val_loss did not improve from 0.09723
Epoch 257/25000

Epoch 00257: val_loss did not improve from 0.09723
Epoch 258/25000

Epoch 00258: val_loss did not improve from 0.09723
Epoch 259/25000

Epoch 00259: val_loss did not improve from 0.09723
Epoch 260/25000

Epoch 00260: val_loss did not improve from 0.09723
Epoch 261/25000

Epoch 00261: val_loss did not improve from 0.09723
Epoch 262/25000

Epoch 00262: val_loss did not improve from 0.09723
Epoch 263/25000

Epoch 00263: val_loss did not improve from 0.09723
Epoch 264/25000

Epoch 00264: val_loss did not improve from 0.09723
Epoch 265/25000

Epoch 00265: val_loss did not improve from 0.09723
Epoch 266/25000

Epoch 00266: val_loss did not improve from 0.09723
Epoch 267/25000

Epoch 00267: val_loss did not i

Epoch 51/25000

Epoch 00051: val_loss improved from 0.08911 to 0.08700, saving model to best_dnn.h5
Epoch 52/25000

Epoch 00052: val_loss improved from 0.08700 to 0.08457, saving model to best_dnn.h5
Epoch 53/25000

Epoch 00053: val_loss improved from 0.08457 to 0.08089, saving model to best_dnn.h5
Epoch 54/25000

Epoch 00054: val_loss did not improve from 0.08089
Epoch 55/25000

Epoch 00055: val_loss did not improve from 0.08089
Epoch 56/25000

Epoch 00056: val_loss did not improve from 0.08089
Epoch 57/25000

Epoch 00057: val_loss did not improve from 0.08089
Epoch 58/25000

Epoch 00058: val_loss did not improve from 0.08089
Epoch 59/25000

Epoch 00059: val_loss did not improve from 0.08089
Epoch 60/25000

Epoch 00060: val_loss did not improve from 0.08089
Epoch 61/25000

Epoch 00061: val_loss did not improve from 0.08089
Epoch 62/25000

Epoch 00062: val_loss did not improve from 0.08089
Epoch 63/25000

Epoch 00063: val_loss did not improve from 0.08089
Epoch 64/25000

Epoch 00064: v

Epoch 174/25000

Epoch 00174: val_loss did not improve from 0.07571
Epoch 175/25000

Epoch 00175: val_loss did not improve from 0.07571
Epoch 176/25000

Epoch 00176: val_loss did not improve from 0.07571
Epoch 177/25000

Epoch 00177: val_loss did not improve from 0.07571
Epoch 178/25000

Epoch 00178: val_loss did not improve from 0.07571
Epoch 179/25000

Epoch 00179: val_loss did not improve from 0.07571
Epoch 180/25000

Epoch 00180: val_loss did not improve from 0.07571
Epoch 181/25000

Epoch 00181: val_loss did not improve from 0.07571
Epoch 182/25000

Epoch 00182: val_loss did not improve from 0.07571
Epoch 183/25000

Epoch 00183: val_loss did not improve from 0.07571
Epoch 184/25000

Epoch 00184: val_loss did not improve from 0.07571
Epoch 185/25000

Epoch 00185: val_loss did not improve from 0.07571
Epoch 186/25000

Epoch 00186: val_loss did not improve from 0.07571
Epoch 187/25000

Epoch 00187: val_loss did not improve from 0.07571
Epoch 188/25000

Epoch 00188: val_loss did not i

Epoch 302/25000

Epoch 00302: val_loss did not improve from 0.07571
Epoch 303/25000

Epoch 00303: val_loss did not improve from 0.07571
Epoch 304/25000

Epoch 00304: val_loss did not improve from 0.07571
Epoch 305/25000

Epoch 00305: val_loss did not improve from 0.07571
Epoch 306/25000

Epoch 00306: val_loss did not improve from 0.07571
Epoch 307/25000

Epoch 00307: val_loss did not improve from 0.07571
Epoch 308/25000

Epoch 00308: val_loss did not improve from 0.07571
Epoch 309/25000

Epoch 00309: val_loss did not improve from 0.07571
Epoch 310/25000

Epoch 00310: val_loss did not improve from 0.07571
Epoch 311/25000

Epoch 00311: val_loss did not improve from 0.07571
Epoch 312/25000

Epoch 00312: val_loss did not improve from 0.07571
Epoch 313/25000

Epoch 00313: val_loss did not improve from 0.07571
Epoch 314/25000

Epoch 00314: val_loss did not improve from 0.07571
Epoch 315/25000

Epoch 00315: val_loss did not improve from 0.07571
Epoch 316/25000

Epoch 00316: val_loss did not i

In [52]:
dnn_pred2 = np.zeros((target.shape[0]))

In [53]:
for tr_idx, val_idx in kf.split(X, y2) :
    tr_x, tr_y = X.iloc[tr_idx], y2.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y2.iloc[val_idx]

    model = Sequential()
    model.add(Dense(512, activation = 'swish'))
    model.add(Dense(256, activation = 'swish'))
    model.add(Dropout(.2))
    model.add(Dense(128, activation = 'swish'))
    model.add(Dense(64, activation = 'swish'))
    model.add(Dropout(.2))
    model.add(Dense(8, activation = 'swish'))
    model.add(Dense(1, activation = 'linear'))

    model.compile(optimizer = Adam(learning_rate = .0003), loss = 'mse')

    es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 300, verbose = 100)
    mc = ModelCheckpoint('best_dnn.h5', mode = 'min', monitor = 'val_loss', save_best_only = True, verbose = 100)

    history = model.fit(tr_x, tr_y, epochs = 25000, verbose = 100, batch_size = 16, callbacks = [es, mc], validation_data = (val_x, val_y))
    best_model = load_model('best_dnn.h5')

    fold_pred = best_model.predict(target) / 15

    dnn_pred2 += [p[0] for p in fold_pred]

Epoch 1/25000

Epoch 00001: val_loss improved from inf to 1.48547, saving model to best_dnn.h5
Epoch 2/25000

Epoch 00002: val_loss improved from 1.48547 to 1.21249, saving model to best_dnn.h5
Epoch 3/25000

Epoch 00003: val_loss improved from 1.21249 to 0.85788, saving model to best_dnn.h5
Epoch 4/25000

Epoch 00004: val_loss improved from 0.85788 to 0.43203, saving model to best_dnn.h5
Epoch 5/25000

Epoch 00005: val_loss improved from 0.43203 to 0.07539, saving model to best_dnn.h5
Epoch 6/25000

Epoch 00006: val_loss did not improve from 0.07539
Epoch 7/25000

Epoch 00007: val_loss did not improve from 0.07539
Epoch 8/25000

Epoch 00008: val_loss improved from 0.07539 to 0.03736, saving model to best_dnn.h5
Epoch 9/25000

Epoch 00009: val_loss improved from 0.03736 to 0.02541, saving model to best_dnn.h5
Epoch 10/25000

Epoch 00010: val_loss did not improve from 0.02541
Epoch 11/25000

Epoch 00011: val_loss improved from 0.02541 to 0.01826, saving model to best_dnn.h5
Epoch 12/250

Epoch 120/25000

Epoch 00120: val_loss did not improve from 0.00504
Epoch 121/25000

Epoch 00121: val_loss did not improve from 0.00504
Epoch 122/25000

Epoch 00122: val_loss did not improve from 0.00504
Epoch 123/25000

Epoch 00123: val_loss did not improve from 0.00504
Epoch 124/25000

Epoch 00124: val_loss did not improve from 0.00504
Epoch 125/25000

Epoch 00125: val_loss improved from 0.00504 to 0.00493, saving model to best_dnn.h5
Epoch 126/25000

Epoch 00126: val_loss did not improve from 0.00493
Epoch 127/25000

Epoch 00127: val_loss did not improve from 0.00493
Epoch 128/25000

Epoch 00128: val_loss did not improve from 0.00493
Epoch 129/25000

Epoch 00129: val_loss improved from 0.00493 to 0.00388, saving model to best_dnn.h5
Epoch 130/25000

Epoch 00130: val_loss did not improve from 0.00388
Epoch 131/25000

Epoch 00131: val_loss did not improve from 0.00388
Epoch 132/25000

Epoch 00132: val_loss did not improve from 0.00388
Epoch 133/25000

Epoch 00133: val_loss did not imp

Epoch 244/25000

Epoch 00244: val_loss did not improve from 0.00318
Epoch 245/25000

Epoch 00245: val_loss did not improve from 0.00318
Epoch 246/25000

Epoch 00246: val_loss did not improve from 0.00318
Epoch 247/25000

Epoch 00247: val_loss did not improve from 0.00318
Epoch 248/25000

Epoch 00248: val_loss did not improve from 0.00318
Epoch 249/25000

Epoch 00249: val_loss did not improve from 0.00318
Epoch 250/25000

Epoch 00250: val_loss did not improve from 0.00318
Epoch 251/25000

Epoch 00251: val_loss did not improve from 0.00318
Epoch 252/25000

Epoch 00252: val_loss did not improve from 0.00318
Epoch 253/25000

Epoch 00253: val_loss did not improve from 0.00318
Epoch 254/25000

Epoch 00254: val_loss did not improve from 0.00318
Epoch 255/25000

Epoch 00255: val_loss did not improve from 0.00318
Epoch 256/25000

Epoch 00256: val_loss did not improve from 0.00318
Epoch 257/25000

Epoch 00257: val_loss did not improve from 0.00318
Epoch 258/25000

Epoch 00258: val_loss did not i

Epoch 374/25000

Epoch 00374: val_loss did not improve from 0.00318
Epoch 375/25000

Epoch 00375: val_loss did not improve from 0.00318
Epoch 376/25000

Epoch 00376: val_loss did not improve from 0.00318
Epoch 377/25000

Epoch 00377: val_loss did not improve from 0.00318
Epoch 378/25000

Epoch 00378: val_loss did not improve from 0.00318
Epoch 379/25000

Epoch 00379: val_loss did not improve from 0.00318
Epoch 380/25000

Epoch 00380: val_loss did not improve from 0.00318
Epoch 381/25000

Epoch 00381: val_loss did not improve from 0.00318
Epoch 382/25000

Epoch 00382: val_loss did not improve from 0.00318
Epoch 383/25000

Epoch 00383: val_loss did not improve from 0.00318
Epoch 384/25000

Epoch 00384: val_loss did not improve from 0.00318
Epoch 385/25000

Epoch 00385: val_loss did not improve from 0.00318
Epoch 386/25000

Epoch 00386: val_loss did not improve from 0.00318
Epoch 387/25000

Epoch 00387: val_loss did not improve from 0.00318
Epoch 388/25000

Epoch 00388: val_loss did not i

Epoch 500/25000

Epoch 00500: val_loss did not improve from 0.00212
Epoch 501/25000

Epoch 00501: val_loss did not improve from 0.00212
Epoch 502/25000

Epoch 00502: val_loss did not improve from 0.00212
Epoch 503/25000

Epoch 00503: val_loss did not improve from 0.00212
Epoch 504/25000

Epoch 00504: val_loss did not improve from 0.00212
Epoch 505/25000

Epoch 00505: val_loss did not improve from 0.00212
Epoch 506/25000

Epoch 00506: val_loss did not improve from 0.00212
Epoch 507/25000

Epoch 00507: val_loss did not improve from 0.00212
Epoch 508/25000

Epoch 00508: val_loss did not improve from 0.00212
Epoch 509/25000

Epoch 00509: val_loss did not improve from 0.00212
Epoch 510/25000

Epoch 00510: val_loss did not improve from 0.00212
Epoch 511/25000

Epoch 00511: val_loss did not improve from 0.00212
Epoch 512/25000

Epoch 00512: val_loss did not improve from 0.00212
Epoch 513/25000

Epoch 00513: val_loss did not improve from 0.00212
Epoch 514/25000

Epoch 00514: val_loss did not i

...

Epoch 121/25000

Epoch 00121: val_loss did not improve from 0.01615
Epoch 122/25000

Epoch 00122: val_loss did not improve from 0.01615
Epoch 123/25000

Epoch 00123: val_loss did not improve from 0.01615
Epoch 124/25000

Epoch 00124: val_loss did not improve from 0.01615
Epoch 125/25000

Epoch 00125: val_loss did not improve from 0.01615
Epoch 126/25000

Epoch 00126: val_loss did not improve from 0.01615
Epoch 127/25000

Epoch 00127: val_loss did not improve from 0.01615
Epoch 128/25000

Epoch 00128: val_loss did not improve from 0.01615
Epoch 129/25000

Epoch 00129: val_loss did not improve from 0.01615
Epoch 130/25000

Epoch 00130: val_loss did not improve from 0.01615
Epoch 131/25000

Epoch 00131: val_loss did not improve from 0.01615
Epoch 132/25000

Epoch 00132: val_loss did not improve from 0.01615
Epoch 133/25000

Epoch 00133: val_loss did not improve from 0.01615
Epoch 134/25000

Epoch 00134: val_loss did not improve from 0.01615
Epoch 135/25000

Epoch 00135: val_loss did not i

Epoch 250/25000

Epoch 00250: val_loss did not improve from 0.01615
Epoch 251/25000

Epoch 00251: val_loss did not improve from 0.01615
Epoch 252/25000

Epoch 00252: val_loss did not improve from 0.01615
Epoch 253/25000

Epoch 00253: val_loss did not improve from 0.01615
Epoch 254/25000

Epoch 00254: val_loss did not improve from 0.01615
Epoch 255/25000

Epoch 00255: val_loss did not improve from 0.01615
Epoch 256/25000

Epoch 00256: val_loss did not improve from 0.01615
Epoch 257/25000

Epoch 00257: val_loss did not improve from 0.01615
Epoch 258/25000

Epoch 00258: val_loss did not improve from 0.01615
Epoch 259/25000

Epoch 00259: val_loss did not improve from 0.01615
Epoch 260/25000

Epoch 00260: val_loss did not improve from 0.01615
Epoch 261/25000

Epoch 00261: val_loss did not improve from 0.01615
Epoch 262/25000

Epoch 00262: val_loss did not improve from 0.01615
Epoch 263/25000

Epoch 00263: val_loss did not improve from 0.01615
Epoch 264/25000

Epoch 00264: val_loss did not i


Epoch 00059: val_loss did not improve from 0.00674
Epoch 60/25000

Epoch 00060: val_loss did not improve from 0.00674
Epoch 61/25000

Epoch 00061: val_loss improved from 0.00674 to 0.00336, saving model to best_dnn.h5
Epoch 62/25000

Epoch 00062: val_loss improved from 0.00336 to 0.00327, saving model to best_dnn.h5
Epoch 63/25000

Epoch 00063: val_loss improved from 0.00327 to 0.00314, saving model to best_dnn.h5
Epoch 64/25000

Epoch 00064: val_loss did not improve from 0.00314
Epoch 65/25000

Epoch 00065: val_loss did not improve from 0.00314
Epoch 66/25000

Epoch 00066: val_loss did not improve from 0.00314
Epoch 67/25000

Epoch 00067: val_loss did not improve from 0.00314
Epoch 68/25000

Epoch 00068: val_loss improved from 0.00314 to 0.00289, saving model to best_dnn.h5
Epoch 69/25000

Epoch 00069: val_loss did not improve from 0.00289
Epoch 70/25000

Epoch 00070: val_loss did not improve from 0.00289
Epoch 71/25000

Epoch 00071: val_loss did not improve from 0.00289
Epoch 72/250

Epoch 186/25000

Epoch 00186: val_loss did not improve from 0.00153
Epoch 187/25000

Epoch 00187: val_loss did not improve from 0.00153
Epoch 188/25000

Epoch 00188: val_loss did not improve from 0.00153
Epoch 189/25000

Epoch 00189: val_loss did not improve from 0.00153
Epoch 190/25000

Epoch 00190: val_loss did not improve from 0.00153
Epoch 191/25000

Epoch 00191: val_loss did not improve from 0.00153
Epoch 192/25000

Epoch 00192: val_loss did not improve from 0.00153
Epoch 193/25000

Epoch 00193: val_loss did not improve from 0.00153
Epoch 194/25000

Epoch 00194: val_loss did not improve from 0.00153
Epoch 195/25000

Epoch 00195: val_loss did not improve from 0.00153
Epoch 196/25000

Epoch 00196: val_loss did not improve from 0.00153
Epoch 197/25000

Epoch 00197: val_loss did not improve from 0.00153
Epoch 198/25000

Epoch 00198: val_loss did not improve from 0.00153
Epoch 199/25000

Epoch 00199: val_loss did not improve from 0.00153
Epoch 200/25000

Epoch 00200: val_loss did not i

Epoch 313/25000

Epoch 00313: val_loss did not improve from 0.00153
Epoch 314/25000

Epoch 00314: val_loss did not improve from 0.00153
Epoch 315/25000

Epoch 00315: val_loss did not improve from 0.00153
Epoch 316/25000

Epoch 00316: val_loss did not improve from 0.00153
Epoch 317/25000

Epoch 00317: val_loss did not improve from 0.00153
Epoch 318/25000

Epoch 00318: val_loss did not improve from 0.00153
Epoch 319/25000

Epoch 00319: val_loss did not improve from 0.00153
Epoch 320/25000

Epoch 00320: val_loss did not improve from 0.00153
Epoch 321/25000

Epoch 00321: val_loss did not improve from 0.00153
Epoch 322/25000

Epoch 00322: val_loss did not improve from 0.00153
Epoch 323/25000

Epoch 00323: val_loss did not improve from 0.00153
Epoch 324/25000

Epoch 00324: val_loss did not improve from 0.00153
Epoch 325/25000

Epoch 00325: val_loss did not improve from 0.00153
Epoch 326/25000

Epoch 00326: val_loss did not improve from 0.00153
Epoch 327/25000

Epoch 00327: val_loss did not i

***
### Ensemble

In [55]:
ml_pred1 = (etr_pred1 + lr_pred1 + svr_pred1 + cb_pred1) / 4
ml_pred2 = (etr_pred2 + lr_pred2 + svr_pred2 + cb_pred2) / 4

In [57]:
print(etr_pred1)

[3.66565314 3.7923647  3.60936185 3.41313998 2.8395883  2.83215533
 3.61431423 3.34272272 3.30328798 2.82257273 3.30251761 3.26799168
 2.89955038 3.02099322 2.73232419 2.89475462 3.19997799 2.98545542
 2.84937479 2.89455367 2.92539207 3.31695529 2.74054348 3.38362183
 2.58272195 3.28743714 3.41793998 3.4995959  3.61443275 2.98878186
 2.7160681  3.50036627 3.12907646 2.9735971  2.95158654 3.36903563
 3.13981233 2.74200379 3.35807929 2.95132359 3.36903563 3.34272272
 3.0253274  3.35807929 3.02185865 3.20132749 2.89869594 2.41675418
 3.79761543 2.81471413 3.35807929 2.70334208 2.60751207 2.65605689
 3.69772834 2.83303703 3.69683945 2.73086388 3.34777323 2.84529734
 2.95907419 2.6290007  2.95608703 2.92539207 3.26945301 3.36903563
 2.95608703 3.36826526 3.50036627 2.74200379 2.73791652 3.12907646
 2.90655813 2.50386749 2.83655742]


In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['sepal width (cm)'] = (ml_pred1 + dnn_pred1) / 2
submission['petal width (cm)'] = (ml_pred2 + dnn_pred2) / 2

In [ ]:
submission.to_csv("dnn_ensemble.csv", index = False)